In [ ]:
from covidvu.utils import autoReloadCode; autoReloadCode()

In [ ]:
from covidvu.cryostation import Cryostation

In [ ]:
import pandas as pd

In [ ]:
from covidvu.predict import _getCountries, buildLogisticModel, predictRegions

In [ ]:
_getCountries('./database/virustrack.db')

In [ ]:
logGrowthModel = buildLogisticModel()

In [ ]:
import os

In [ ]:
TEST_SITE_DATA = os.path.join(os.getcwd(), 'resources', 'test_site_data')

In [ ]:
predictRegions('all',
               regionType='country',
               nDaysPredict=10,
               siteData=TEST_SITE_DATA,
               logGrowthModel=logGrowthModel,
               limitRegions=3,
               )